# 诗歌生成

# 数据处理

In [19]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets

start_token = 'bos'
end_token = 'eos'

def process_dataset(fileName):
    examples = []
    with open(fileName, 'r',encoding='utf-8') as fd:
        for line in fd:
            outs = line.strip().split(':')
            content = ''.join(outs[1:])
            ins = [start_token] + list(content) + [end_token] 
            if len(ins) > 200:
                continue
            examples.append(ins)
            
    counter = collections.Counter()
    for e in examples:
        for w in e:
            counter[w]+=1
    
    sorted_counter = sorted(counter.items(), key=lambda x: -x[1])  # 排序
    words, _ = zip(*sorted_counter)
    words = ('PAD', 'UNK') + words[:len(words)]
    word2id = dict(zip(words, range(len(words))))
    id2word = {word2id[k]:k for k in word2id}
    
    indexed_examples = [[word2id[w] for w in poem]
                        for poem in examples]
    seqlen = [len(e) for e in indexed_examples]
    
    instances = list(zip(indexed_examples, seqlen))
    
    return instances, word2id, id2word

def poem_dataset():
    instances, word2id, id2word = process_dataset('./poems.txt')
    ds = tf.data.Dataset.from_generator(lambda: [ins for ins in instances], 
                                            (tf.int64, tf.int64), 
                                            (tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.shuffle(buffer_size=10240)
    ds = ds.padded_batch(100, padded_shapes=(tf.TensorShape([None]),tf.TensorShape([])))
    ds = ds.map(lambda x, seqlen: (x[:, :-1], x[:, 1:], seqlen-1))
    return ds, word2id, id2word

In [20]:
ds,word2id,id2word = poem_dataset()

# 模型代码， 完成建模代码

In [21]:
class myRNNModel(keras.Model):
    def __init__(self, w2id):
        super(myRNNModel, self).__init__()
        self.v_sz = len(w2id)
        self.embed_layer = tf.keras.layers.Embedding(self.v_sz, 64, 
                                                    batch_input_shape=[None, None])  # one-shot编码，总共v_sz大小
        
        self.rnncell = tf.keras.layers.SimpleRNNCell(128)
        self.rnn_layer = tf.keras.layers.RNN(self.rnncell, return_sequences=True)
        self.dense = tf.keras.layers.Dense(self.v_sz)
        
    @tf.function
    def call(self, inp_ids):
        '''
        此处完成建模过程，可以参考Learn2Carry
        '''
        inp_emb = self.embed_layer(inp_ids)
        inp_out = self.rnn_layer(inp_emb)
        logits = self.dense(inp_out)
        return logits
    
    @tf.function
    def get_next_token(self, x, state):
        '''
        shape(x) = [b_sz,] 
        '''
        inp_emb = self.embed_layer(x) #shape(b_sz, emb_sz)
        h, state = self.rnncell.call(inp_emb, state) # shape(b_sz, h_sz)
        logits = self.dense(h) # shape(b_sz, v_sz)
        out = tf.argmax(logits, axis=-1)
        return out, state

## 一个计算sequence loss的辅助函数，只需了解用途。

In [22]:
def mkMask(input_tensor, maxLen):
    shape_of_input = tf.shape(input_tensor)
    shape_of_output = tf.concat(axis=0, values=[shape_of_input, [maxLen]])

    oneDtensor = tf.reshape(input_tensor, shape=(-1,))
    flat_mask = tf.sequence_mask(oneDtensor, maxlen=maxLen)
    return tf.reshape(flat_mask, shape_of_output)


def reduce_avg(reduce_target, lengths, dim):
    """
    Args:
        reduce_target : shape(d_0, d_1,..,d_dim, .., d_k)
        lengths : shape(d0, .., d_(dim-1))
        dim : which dimension to average, should be a python number
    """
    shape_of_lengths = lengths.get_shape()
    shape_of_target = reduce_target.get_shape()
    if len(shape_of_lengths) != dim:
        raise ValueError(('Second input tensor should be rank %d, ' +
                         'while it got rank %d') % (dim, len(shape_of_lengths)))
    if len(shape_of_target) < dim+1 :
        raise ValueError(('First input tensor should be at least rank %d, ' +
                         'while it got rank %d') % (dim+1, len(shape_of_target)))

    rank_diff = len(shape_of_target) - len(shape_of_lengths) - 1
    mxlen = tf.shape(reduce_target)[dim]
    mask = mkMask(lengths, mxlen)
    if rank_diff!=0:
        len_shape = tf.concat(axis=0, values=[tf.shape(lengths), [1]*rank_diff])
        mask_shape = tf.concat(axis=0, values=[tf.shape(mask), [1]*rank_diff])
    else:
        len_shape = tf.shape(lengths)
        mask_shape = tf.shape(mask)
    lengths_reshape = tf.reshape(lengths, shape=len_shape)
    mask = tf.reshape(mask, shape=mask_shape)

    mask_target = reduce_target * tf.cast(mask, dtype=reduce_target.dtype)

    red_sum = tf.reduce_sum(mask_target, axis=[dim], keepdims=False)
    red_avg = red_sum / (tf.cast(lengths_reshape, dtype=tf.float32) + 1e-30)
    return red_avg

# 定义loss函数，定义训练函数

In [23]:
@tf.function
def compute_loss(logits, labels, seqlen):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    losses = reduce_avg(losses, seqlen, dim=1)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, seqlen):
    '''
    完成一步优化过程，可以参考之前做过的模型
    '''
    with tf.GradientTape() as tape:
        logits = model.call(x)
        loss = compute_loss(logits,y,seqlen)
    
    grad = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grad,model.trainable_variables))
    return loss

def train(epoch, model, optimizer, ds):
    loss = 0.0
    accuracy = 0.0
    for step, (x, y, seqlen) in enumerate(ds):
        loss = train_one_step(model, optimizer, x, y, seqlen)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy())

    return loss

# 训练优化过程

In [24]:
optimizer = optimizers.Adam(0.0005)
train_ds, word2id, id2word = poem_dataset()
model = myRNNModel(word2id)

for epoch in range(10):
    loss = train(epoch, model, optimizer, train_ds)

epoch 0 : loss 8.819718
epoch 1 : loss 6.529677
epoch 2 : loss 6.0968494
epoch 3 : loss 5.8768816
epoch 4 : loss 5.721767
epoch 5 : loss 5.4939103
epoch 6 : loss 5.4974537
epoch 7 : loss 5.3521833
epoch 8 : loss 5.3929687
epoch 9 : loss 5.3344994


# 生成过程

In [60]:
def gen_sentence(start_word="bos"):
    state = [tf.random.normal(shape=(1, 128), stddev=0.5), tf.random.normal(shape=(1, 128), stddev=0.5)]
    cur_token = tf.constant([word2id[start_word]], dtype=tf.int32)
    collect = [cur_token.numpy()[0]]
    for _ in range(50):
        cur_token, state = model.get_next_token(cur_token, state)
        collect.append(cur_token.numpy()[0])
    return [id2word[t] for t in collect if t!=0 and t!=1 and t!=4 and t!=5 and t!=975 and t!=976 and t!=1025 and t!=1026]


print(''.join(gen_sentence("日")))
print(''.join(gen_sentence("红")))
print(''.join(gen_sentence("山")))
print(''.join(gen_sentence("夜")))
print(''.join(gen_sentence("湖")))
print(''.join(gen_sentence("海")))
print(''.join(gen_sentence("月")))
print(''.join(gen_sentence("春")))
print(''.join(gen_sentence("冬")))
print(''.join(gen_sentence("君")))
print(''.join(gen_sentence()))

日暮。君不得人间事，不得人间不可知。道不知君不得，不知何处到春风。君不是无人事，不是东山不可知。
红蓉畔黏湘畔柳条春雨，花声满树，风吹一枝。来不见，不知此人。之不得，不知何事。之不得，不知何事。
山畔水，一片花声。风吹落，风风。云不见，不知此人。德，不知何事。德，不知有人。德
夜向风声。有人间去，何人不可知。君无所见，不见此人间。有无人事，无人不可知。君无所见，不见此人
湖边月落月，风雨满花声。有无人事，何人不可知。君无所见，不见此人间。有无人事，无人不可知。君无
海上。君不得，不知此人。德，不知何事。德，不知有人。德，不以之之之兮，何事，
月，相思有人，不知何处。君不得，不知何事。之不足，不知何事。之不得，不知何事。之不得，不知何事
春风吹落，风吹落月深。风无处处，山雨落云声。马无人去，无人不可知。君无所见，不见此人间。有无人
冬里。公无事不相思，不得人间不可知。道不知君不得，不知何处到春风。君不是无人事，不是东山不可知。
君子向江头，不得长家得。有一时人，不知人不得。君不可知，不见人间心。然不可见，不得人间心。有不
公不是三年事，不得人间不可知。君不得人间事，不是东山不可知。道不知君不得，不知何处到春风。君
